# 신장의무기록사본 OCR 서비스 구현

# 1. 라이브러리 로드

In [1]:
from glob import glob
import pandas as pd
import cv2
import numpy as np
import math

from tensorflow.keras.models import load_model
import efficientnet.tfkeras # for swish
import tensorflow as tf
import json

import os
os.chdir('D:/kidney/source')

# 2. 모델 로드

In [2]:
model = load_model('../model/OCR_ver1.0.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# 3. 예측 준비 작업

### 3-1. 함수

In [3]:
def border_make(image):
    row, col = image.shape[:2]
    bottom = image[row-2:row, 0:col]
    mean = cv2.mean(bottom)[0]
    
    col_bordersize = (55-col)/2
    row_bordersize = (55-row)/2
    
    border = cv2.copyMakeBorder(
        image,
        top = math.ceil(row_bordersize),
        bottom = math.floor(row_bordersize),
        left = math.ceil(col_bordersize),
        right = math.floor(col_bordersize),
        borderType = cv2.BORDER_ISOLATED, #BORDER_ISOLATED  BORDER_CONSTANT
        value = [mean, mean, mean]
    )
    return border

### 3-2. Y_test

In [4]:
# 예측 데이터 레이블 값 #Y_test # actual_value_lst
Y_test = pd.read_csv('../data/Y_test.csv', encoding='utf-8-sig') #Y_test = ['.','1','1','8','1','0','2','3','/','희','찬','0','자','출','명','증','본','의','력','다','합','을','음','없','림','틀','과','의','록','무','은','니','본','사','한','부','첨','~','~','~','~']
Y_test = list(Y_test['col_names'])

pd.DataFrame(data = Y_test, columns=['col_names']).to_csv('../data/Y_test.csv', index = False, encoding='utf-8-sig')

### 3-3. Y_train_unique

In [5]:
Y_train_unique = pd.read_csv('../data/Y_train_unique.csv', encoding= 'utf-8-sig').loc[:, '0']  # list # 122        # Y_train_columns = Y_train_columns.loc[:, '0']         # Y_train_columns > Y_train_unique 

# 4. 예측 데이터 불러오기

In [6]:
predict_lst = glob('D:/kidney/image/forPredict/*.png') 

In [7]:
img_lst = []
for predict_lst_num in range(len(predict_lst)):
    image = cv2.imread(predict_lst[predict_lst_num], cv2.IMREAD_COLOR)
    img = border_make(image)
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret ,img = cv2.threshold(dst,127,255,0)
    img_lst.append(img)
    img_array = np.array(img_lst)
img_array_reshape = img_array.reshape(-1,55,55,1)
X_test = img_array_reshape
X_test.shape

(41, 55, 55, 1)

# 5. 예측

### 5-1. 예측 확률을 데이터프레임 형태로 출력

In [29]:
pred = model.predict(X_test) # 41 
#pred_df = pd.DataFrame(pred).round(2) #(41, 122)
pred_df = pd.DataFrame(pred)#.round(2) #(41, 122)
pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
0,4.018439e-03,4.012331e-03,1.875072e-03,4.730043e-03,3.105285e-02,0.087583,3.998162e-02,0.009099,0.035988,4.684531e-03,...,6.197036e-03,2.138751e-03,7.144580e-03,3.532618e-03,3.543430e-03,1.008884e-02,3.532248e-03,4.076332e-03,2.534816e-03,1.328062e-03
1,3.440813e-10,2.024830e-02,3.075334e-16,4.972157e-08,1.159422e-04,0.000017,4.488513e-14,0.000056,0.745508,1.287353e-10,...,1.682235e-12,4.915690e-07,5.469886e-15,1.059483e-09,4.938166e-10,3.891970e-09,2.218907e-12,4.229805e-09,8.172111e-09,2.062717e-09
2,6.286983e-04,1.368758e-03,9.797497e-03,8.863497e-05,1.316238e-03,0.000275,7.035227e-05,0.070739,0.290929,1.980577e-04,...,1.651043e-04,1.092544e-04,2.673087e-04,2.708556e-05,3.540422e-03,1.118313e-02,7.568792e-06,1.202097e-04,2.122431e-04,4.652883e-04
3,4.444835e-06,3.779002e-13,3.180779e-06,1.244283e-04,2.455803e-07,0.000002,2.753349e-04,0.005923,0.010731,1.745171e-01,...,2.941164e-10,5.525878e-09,9.831428e-15,1.858534e-07,2.311413e-07,2.016371e-13,1.770037e-14,1.979940e-10,1.717215e-07,6.887475e-13
4,1.128022e-09,7.999078e-04,3.734640e-16,1.197654e-08,3.292283e-05,0.000002,1.181726e-13,0.000009,0.963554,1.973100e-11,...,2.087074e-13,6.305231e-07,3.007527e-15,4.991607e-11,4.465361e-10,4.580045e-09,3.145127e-13,6.694153e-10,2.604812e-09,1.239520e-10


### 5-1-1. 실제와 예측 값 간단하게 비교

In [30]:
### 최고 확률 컬럼 알아내기

pred_lst = [] #list # 41
for row in range(len(pred_df)):
    #pred_col_num = pred_df.loc[:,pred_df.loc[row] == round(pred_df.iloc[row],2).max()].columns[0]
    pred_col_num = pred_df.loc[:,pred_df.loc[row] == pred_df.iloc[row].max()].columns[0]
    pred_lst.append(pred_col_num)

pd.DataFrame(data= pred_df.iloc[0,:]).to_csv('../data/testtesttest.csv')

In [12]:
### 가장 높게 예측된 확률을 label로 바꾸기

predict_lst = [] # list # 41
for i, column_num in enumerate(pred_lst):
    predict_lst.append(Y_train_unique[column_num])

In [49]:
### 최고 확률 컬럼 
probability_lst = []
for row in range(len(pred_df)):
    percentage = round(pred_df.iloc[row,:].max()*100,3)
    probability_lst.append(percentage)

### EDA 추가 코드

In [51]:
### 데이터 프레임 형태로 비교해 보기

result_df = pd.DataFrame(columns=['predict', 'actual', 'probability'])

result_df.actual = Y_test
result_df.predict = predict_lst
result_df.probability = probability_lst
result_df# 숫자는 나름 잘 맞추는거 같은데 한글은 의무기록지 한장으로는 통 못 맞춤

,predict,actual,probability
0,.,.,8.758
1,1,1,74.551
2,1,1,29.093
3,x,8,50.648
4,1,1,96.355
5,0,0,79.204
6,1,2,94.778
7,3,3,85.731
8,/,/,70.107
9,1,희,90.826


# 7. 모델 평가

In [13]:
# 다중 레이블 다중 분류

### TP, TN, FP, FN  //  accuracy, precision, recall, F1

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

acc = accuracy_score(Y_test, predict_lst)

precision_none = precision_score(Y_test, predict_lst, average=None)
precision_micro = precision_score(Y_test, predict_lst, average='micro')
precision_macro = precision_score(Y_test, predict_lst, average='macro') # 'micro', 'macro', 'weighted'
precision_weighted = precision_score(Y_test, predict_lst, average='weighted') # 'micro', 'macro', 'weighted'

recall_micro = recall_score(Y_test, predict_lst, average='micro')
recall_macro = recall_score(Y_test, predict_lst, average='macro') # 'micro', 'macro', 'weighted'
recall_weighted = recall_score(Y_test, predict_lst, average='weighted') # 'micro', 'macro', 'weighted'

f1_weighted = f1_score(Y_test, predict_lst, labels=None, average="weighted")

print('accurracy: ', acc)

print('precision_none: ', precision_none)
print('precision_micro: ', precision_micro)
print('precision_macro: ', precision_macro)
print('precision_weighted: ', precision_weighted)

print('recall_micro: ', recall_micro)
print('recall_macro: ', recall_macro)
print('recall_weighted: ', recall_weighted)

print('f1_weighted: ', f1_weighted)

accurracy:  0.24390243902439024
precision_none:  [1.         0.5        1.         0.42857143 0.         0.5
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.
 0.         0.         0.         0.         0.5        0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
precision_micro:  0.24390243902439024
precision_macro:  0.10714285714285715
precision_weighted:  0.17770034843205573
recall_micro:  0.24390243902439024
recall_macro:  0.14130434782608695
recall_weighted:  0.24390243902439024
f1_weighted:  0.1983739837398374


# 8. roc커브, auc 구하기

In [54]:
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

C:\Users\SURIMWANG\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\SURIMWANG\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy

In [92]:
print(pred.shape)
print(type(pred))

(41, 122)
<class 'numpy.ndarray'>


실제값을 타이핑 해뒀는데 그게 아니라 어레이로 만들어져야 할듯

어떻게? 잘해봐 왕수

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# 9. need to update labelme